In [1]:
# Paso 1: Importar Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import os

## Cargar los Datos

In [2]:
# Cargar el dataset
data_path = "../data/Telco-Customer-Churn.csv"
df = pd.read_csv(data_path)

# Mostrar las primeras filas
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/Telco-Customer-Churn.csv'

## Análisis Exploratorio de Dato

In [3]:
# Resumen Estadístico:
df.describe()

NameError: name 'df' is not defined

In [4]:
# Distribución de Clases:
import matplotlib.pyplot as plt
df['Churn'].value_counts().plot(kind='bar')
plt.title("Distribución de la variable objetivo (Churn)")
plt.show()

NameError: name 'df' is not defined

In [5]:
# Análisis de Variables Categóricas:
df['gender'].value_counts().plot(kind='bar')
plt.title("Distribución de género")
plt.show()

NameError: name 'df' is not defined

In [6]:
# Correlación entre Variables Numéricas:
import seaborn as sns
corr_matrix = df.corr()
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm")
plt.title("Matriz de correlación")
plt.show()

NameError: name 'df' is not defined

In [7]:
# Histograma de Ingresos:
df['MonthlyCharges'].hist(bins=20)
plt.title("Distribución de Monthly Charges")
plt.show()

NameError: name 'df' is not defined

## Preparación de Datos

In [8]:
## Normalización:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.select_dtypes(include=['float64', 'int64']))

NameError: name 'df' is not defined

In [9]:
# División de Datos:
from sklearn.model_selection import train_test_split
X = df.drop('Churn', axis=1)
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'df' is not defined

## Implementación de KNN

In [10]:
# Entrenamiento del Modelo:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [11]:
# Evaluación:
from sklearn.metrics import accuracy_score
y_pred = knn.predict(X_test)
print("Precisión del modelo:", accuracy_score(y_test, y_pred))

NameError: name 'X_test' is not defined

In [12]:
# Visualización de Precisión para Diferentes Vecinos:
accuracies = []
for k in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracies.append(accuracy_score(y_test, y_pred))

plt.plot(range(1, 21), accuracies)
plt.title("Precisión vs Número de Vecinos")
plt.xlabel("Número de Vecinos")
plt.ylabel("Precisión")
plt.show()

NameError: name 'X_train' is not defined